In [1]:
import matplotlib.pyplot as plt
import os
from datetime import datetime
import time
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc, recall_score, roc_auc_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch


### Datas preparation

In [2]:
def csv_data_into_bunch(filenames):
    """
    This function assume that the csv file will have label egal to 1 for anomalie and 0 for normal value.
    The csv have a header too.
    """
    datasets = []
    for i in filenames:
        name = os.path.splitext(os.path.basename(i))[0]

        data = pd.read_csv(i).to_numpy()
    
        x = data[:,:-1]
        y = data[:,-1]

        # Standardisation
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)

        # Store in a Bunch
        datasets.append(Bunch(name=name, data=x_scaled, target=y))

    return datasets


        
def list_all_file_name_in_folder(folder="../../data_rapport"):
    return [os.path.join(folder, filename) for filename in os.listdir(folder) if filename.endswith(".csv")]

filenames = list_all_file_name_in_folder()
print(filenames)

datasets = csv_data_into_bunch(filenames)
speech = datasets[0]

['../../data_rapport/speech.csv', '../../data_rapport/cancerS.csv', '../../data_rapport/donneurs.csv', '../../data_rapport/satellite.csv', '../../data_rapport/fraude.csv', '../../data_rapport/http.csv', '../../data_rapport/shuttle.csv']


### Benchmark

This function takes two parameters: `datasets` and `algorithms`.

- `datasets`: a list of `Bunch` objects, each containing:
  - `name`: the name of the dataset  
  - `data`: the datas  
  - `target`: the label 



- `algorithms`: a list of tuples in the following form:

  ```python
  algorithms = [
      ("name", # Algorithm name 
       run_name, # runner function
       {"param1": ..., "param2": ...}), # dict of paramaters 
      # …
  ]

This function will return three pandas dataFramme : auc, recall, time

In [3]:
def benchmark(datasets, algorithms):
    """
    Run a benchmark of multiple algorithms on multiple datasets.
    
     datasets : list of Bunch
            Each Bunch must have attributes:
              - name : dataset name
              - data : feature data
              - target : ground truth labels
        algorithms : list of tuples
            Each tuple is (algo_name, runner_function, params_dict):
              - algo_name : descriptive name of the algorithm
              - runner_function : function(bunch, params) - > dict
              - params_dict: parameters to pass to the runner


    Returns
    -------
    df_auc : pandas.DataFrame
        AUC scores for each (dataset × algorithm).
    df_recall : pandas.DataFrame
        Recall scores for each (dataset × algorithm).
    df_time : pandas.DataFrame
        Execution time (seconds) for each (dataset × algorithm).
    """
    
    auc_list, recall_list, time_list = [], [], []
    
    for bunch in datasets:
        print(bunch.name, " : départ")
        auc_info, recall_info, time_info = {"dataset": bunch.name}, {"dataset": bunch.name}, {"dataset": bunch.name}
        
        for algo_name, algo_runner, params in algorithms:
            print("  ", algo_name, ": départ")
            res = algo_runner(bunch, params) 
            
            auc_info[algo_name]    = res['auc']
            recall_info[algo_name] = res['recall']
            time_info[algo_name]   = res['time']
            
            print("AUC :", res['auc'], "RAPPEL :", res['recall'],"TEMPS :", res['time'])
            print("  ", algo_name, ": fini")
            
        auc_list.append(auc_info)
        recall_list.append(recall_info)
        time_list.append(time_info)
        
        print(bunch.name, " : fini")
        
    df_auc    = pd.DataFrame(auc_list)
    df_recall = pd.DataFrame(recall_list)
    df_time   = pd.DataFrame(time_list)
    
    return df_auc, df_recall, df_time
        

### Algorithms 

Each runner function must accept exactly two arguments and return a dictionary with the same keys:


- `Parameters`

    - `bunch` : like in the datasets list of the benchmark parameters
    
    - `params` :  like in the algorithms list of the benchmark parameters

- `Returns`

- `dict`  
  ```python
  {
    "auc":    <float>,  // ROC AUC score
    "recall": <float>,  // Recall score
    "time":   <float>   // Elapsed time (seconds)
  }


In [4]:
def run_iforest(bunch, params):
    """
    Fit and evaluate IsolationForest on a single dataset.

    Parameters
    ----------
    bunch : Bunch
        Must have attributes:
          - data: feature data
          - target: ground truth labels
    params : dict
        Keyword arguments for IsolationForest constructor.

    Returns
    -------
    dict
        {
          'auc': float,      # ROC AUC score
          'recall': float,   # recall score
          'time': float      # elapsed time in seconds
        }
    """
    
    x, y_ground_truth = bunch.data, bunch.target
    IF = IsolationForest(**params)
    
    # Fit + predict 
    start = time.perf_counter()
    IF.fit(x)
    y_pred = IF.predict(x) 

    elapsed = time.perf_counter() - start

    # Binairisation of the prediction and scores calculation 
    y_pred = (y_pred == -1)
    scores = -IF.decision_function(x)


    # metrics
    auc = roc_auc_score(y_ground_truth, scores)
    recall = recall_score(y_ground_truth, y_pred)

    return {
        'auc': auc,
        'recall': recall,
        'time' : elapsed
    }


def run_lof(bunch, params) :
    """
    Fit and evaluate LocalOutlierFactor on a single dataset.

    Parameters
    ----------
    bunch : Bunch
        Must have attributes:
          - data: feature data
          - target: ground truth labels
    params : dict
        Keyword arguments for LocalOutlierFactor constructor.

    Returns
    -------
    dict
        {
          'auc': float,      # ROC AUC score
          'recall': float,   # recall score
          'time': float      # elapsed time in seconds
        }
    """
    x, y_ground_truth = bunch.data, bunch.target
    LOF = LocalOutlierFactor(** params)
    
    # Fit + predict 
    start = time.perf_counter()
    y_pred = LOF.fit_predict(x)
    elapsed = time.perf_counter() - start

    # Binairisation of the prediction and scores calculation 
    y_pred = (y_pred == -1)
    scores = -LOF.negative_outlier_factor_

    # metrics
    auc = roc_auc_score(y_ground_truth, scores)
    recall = recall_score(y_ground_truth, y_pred)

    return {
        'auc': auc,
        'recall': recall,
        'time' : elapsed
    }

In [5]:
algorithms = [
     ("IForest", run_iforest, {}),
     ("LOF", run_lof, {}),
]


#df_auc, df_recall, df_time = benchmark(datasets, algorithms)
#df_auc, df_recall, df_time = benchmark([speech], algorithms)

In [6]:
def add_hlines(latex):
    return latex.replace("\\\\", "\\\\ \\hline")
        
def pandas_to_latex(df_auc, df_recall, df_time):
    """
    Convert three pandas DataFrames into LaTeX-formatted tables with horizontal lines.

    Parameters
    ----------
    df_auc : pandas.DataFrame
        DataFrame of AUC scores.
    df_recall : pandas.DataFrame
        DataFrame of recall scores.
    df_time : pandas.DataFrame
        DataFrame of execution times.

    Returns
    -------
    tuple of str
        A tuple (latex_auc, latex_recall, latex_time), where each element is the
        LaTeX code for the corresponding DataFrame, with added \\hline commands.
    """
    latex_auc = add_hlines(df_auc.to_latex(index=False,float_format="%.2f"))
    latex_recall = add_hlines(df_recall.to_latex(index=False,float_format="%.2f"))
    latex_time = add_hlines(df_time.to_latex(index=False,float_format="%.2f"))

    return latex_auc, latex_recall, latex_time

#latex_auc, latex_recall, latex_time = pandas_to_latex(df_auc, df_recall, df_time)



In [7]:
def write_latex_in_file(latex_auc, latex_recall,latex_time, algorithms, filepath = "./latex_results.txt"):
    """
    Prepend LaTeX tables and algorithm info to a text file with a timestamp.

    Parameters
    ----------
    latex_auc : str
        LaTeX code for the AUC table.
    latex_recall : str
        LaTeX code for the recall table.
    latex_time : str
        LaTeX code for the time table.
    algorithms : list of tuples
        Same list passed to benchmark, each tuple is
        (name, runner_function, params_dict).
    filepath : str, optional
        Path to the output text file (default "./latex_results.txt").

    Notes
    -----
    - The current timestamp is written at the top.
    - Algorithm names and their parameter settings are listed.
    - New content is prepended to preserve previous runs.
    """
    # Actual time
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Informations about algorithms used : name1 : param1 = 0, param2=0 ; ...
    infos = []
    for name, _, params in algorithms:
        param_str = ", ".join(f"{k}={v}" for k, v in params.items())
        infos.append(f"{name}: {param_str}")
    info_algo = "; ".join(infos)

    content = (
        f"{now}\n"
        f"{info_algo}\n\n"
        r"%=== AUC table ===" "\n"
        f"{latex_auc}\n\n"
        r"%=== Recall table ===" "\n"
        f"{latex_recall}\n\n"
        r"%=== Time table ===" "\n"
        f"{latex_time}\n"
    )

    # Add at the start of the file without delete the old content 
    old = ""
    if os.path.exists(filepath):
        with open(filepath, "r") as f:
            old = f.read()
        with open(filepath, "w") as f:
            f.write(content)
            f.write(old)
    
    

### Combination 

In [8]:
def automatisatison(datasets, algorithms, filepath = "./latex_results.txt"):
    """
    Run the full pipeline: benchmark algorithms, convert results to LaTeX, and write to file.

    Parameters
    ----------
    datasets : list of Bunch
        List of dataset objects (each with .name, .data, .target).
    algorithms : list of tuples
        Each tuple is (name, runner_function, params_dict), as for benchmark().
    filepath : str, optional
        Path where the LaTeX output will be written (default "./latex_results.txt").

    """
    start = time.perf_counter()
    
    # Calculate all the metrics
    df_auc, df_recall, df_time = benchmark(datasets, algorithms)

    # Transform the metrics into latex tab 
    latex_auc, latex_recall, latex_time = pandas_to_latex(df_auc, df_recall, df_time)

    # Write these latex tab in a file
    write_latex_in_file(latex_auc, latex_recall,latex_time, algorithms, filepath)

    elapsed = time.perf_counter() - start


In [ ]:
automatisation(datasets, algorithms)

### PRINT, CSV, 

python3 benchmark.py 2>&1 | tee prints_and_errors.txt  

In [24]:
def pandas_to_csv(df_auc, df_recall, df_time, name_csv, new_folder_path="./csv_results/"):
    # Creation of a new folder
    os.makedirs(new_folder_path + name_csv, exist_ok=True)
    
    df_auc.to_csv(new_folder_path + name_csv + "/" +name_csv + "_auc.csv", index=False, header=True, float_format='%.2f')
    df_recall.to_csv(new_folder_path + name_csv + "/" +name_csv +"_recall.csv", index=False, header=True,  float_format='%.2f')
    df_time.to_csv(new_folder_path + name_csv + "/" +name_csv +"_time.csv", index=False, header=True, float_format='%.2f')

pandas_to_csv(df_auc, df_recall, df_time, "IF_LOF_Cancer_Speech")

### NaN Values 

```python 
Traceback (most recent call last):
  File "/home/khalil/4eme/stage/tests/benchmark.py", line 483, in <module>
    automatisation(datasets, algorithms)
  File "/home/khalil/4eme/stage/tests/benchmark.py", line 432, in automatisation
    df_auc, df_recall, df_time = benchmark(datasets, algorithms)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/khalil/4eme/stage/tests/benchmark.py", line 308, in benchmark
    res = algo_runner(bunch, params)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/khalil/4eme/stage/tests/benchmark.py", line 184, in run_vae
    VAE_model.fit(x)
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/pyod/models/base_dl.py", line 202, in fit
    self.decision_scores_ = self.decision_function(X)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/pyod/models/base_dl.py", line 290, in decision_function
    anomaly_scores = self.evaluate(data_loader)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/pyod/models/base_dl.py", line 315, in evaluate
    score = self.evaluating_forward(batch_data)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/pyod/models/vae.py", line 264, in evaluating_forward
    score = pairwise_distances_no_broadcast(x.numpy(),
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/pyod/utils/stat_models.py", line 43, in pairwise_distances_no_broadcast
    Y = check_array(Y)
        ^^^^^^^^^^^^^^
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/utils/validation.py", line 1107, in check_array
    _assert_all_finite(
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/utils/validation.py", line 120, in _assert_all_finite
    _assert_all_finite_element_wise(
  File "/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/utils/validation.py", line 169, in _assert_all_finite_element_wise
    raise ValueError(msg_err)
ValueError: Input contains NaN.

In [8]:
for bunch in datasets:
    has_nan = np.isnan(bunch.data).any()
    print(f"{bunch.name} has NaN?  {has_nan}")

speech has NaN?  False
satellite has NaN?  False
fraude has NaN?  False
cancer du sein has NaN?  False
shuttle has NaN?  False


### Benchmark with checkpoints

In [9]:
def create_checkpoint_csv(algorithms, auc_csv, recall_csv,time_csv):
    os.makedirs("./checkpoints", exist_ok=True) # Create the repertories 
    algo_names  = [name for name, _, _ in algorithms]
    cols = ["dataset"] + algo_names
    
    # Create empty CSVs with header row
    pd.DataFrame(columns=cols).to_csv(auc_csv,  mode="w",  index=False)
    pd.DataFrame(columns=cols).to_csv(recall_csv, mode="w", index=False)
    pd.DataFrame(columns=cols).to_csv(time_csv, mode="w",  index=False)

def append_dict_to_csv(row, path): 
    """
        Add a row in a csv file, I use pandas because it's easy to write and I have a small number of dataset
    """
    pd.DataFrame([row]).to_csv(path, mode="a", header=False, index=False, float_format="%.2f")


In [12]:
create_checkpoint_csv(algorithms,"./checkpoints/auc_checkpoint.csv", "./checkpoints/recall_checkpoint.csv","./checkpoints/time_checkpoint.csv")

In [ ]:
def benchmark_checkpoints(datasets, algorithms, auc_csv="./checkpoints/auc_checkpoint.csv", recall_csv="./checkpoints/recall_checkpoint.csv",
              time_csv="./checkpoints/time_checkpoint.csv"):
    """
    Run a benchmark of multiple algorithms on multiple datasets.
    
     datasets : list of Bunch
            Each Bunch must have attributes:
              - name : dataset name
              - data : feature data
              - target : ground truth labels
        algorithms : list of tuples
            Each tuple is (algo_name, runner_function, params_dict):
              - algo_name : descriptive name of the algorithm
              - runner_function : function(bunch, params) - > dict
              - params_dict: parameters to pass to the runner


    Returns
    -------
    df_auc : pandas.DataFrame
        AUC scores for each (dataset × algorithm).
    df_recall : pandas.DataFrame
        Recall scores for each (dataset × algorithm).
    df_time : pandas.DataFrame
        Execution time (seconds) for each (dataset × algorithm).
    """

    create_checkpoint_csv(algorithms, auc_csv, recall_csv,time_csv) ### <- <-
    
    auc_list, recall_list, time_list = [], [], []
    
    for bunch in datasets:
        print(bunch.name, " : départ")
        auc_info, recall_info, time_info = {"dataset": bunch.name}, {"dataset": bunch.name}, {"dataset": bunch.name}
        
        for algo_name, algo_runner, params in algorithms:
            print("  ", algo_name, ": départ")
            res = algo_runner(bunch, params) 
            
            auc_info[algo_name]    = res['auc']
            recall_info[algo_name] = res['recall']
            time_info[algo_name]   = res['time']
            
            print("AUC :", res['auc'], "RAPPEL :", res['recall'],"TEMPS :", res['time'])
            print("  ", algo_name, ": fini")
            
        auc_list.append(auc_info)
        recall_list.append(recall_info)
        time_list.append(time_info)

        append_dict_to_csv(auc_info,auc_csv) ### <- <-
        append_dict_to_csv(recall_info,recall_csv)
        append_dict_to_csv(time_info,time_csv)
           
        print(bunch.name, " : fini")

    df_auc    = pd.DataFrame(auc_list)
    df_recall = pd.DataFrame(recall_list)
    df_time   = pd.DataFrame(time_list)
    
    return df_auc, df_recall, df_time


benchmark_checkpoints(datasets, algorithms)

speech  : départ
   IForest : départ
AUC : 0.47671226681741097 RAPPEL : 0.01639344262295082 TEMPS : 0.1239963070001977
   IForest : fini
   LOF : départ
AUC : 0.5078168456755229 RAPPEL : 0.0 TEMPS : 0.1987326070011477
   LOF : fini
speech  : fini
cancerS  : départ
   IForest : départ
AUC : 0.973109243697479 RAPPEL : 0.8 TEMPS : 0.09633737500189454
   IForest : fini
   LOF : départ
AUC : 0.9845938375350141 RAPPEL : 0.9 TEMPS : 0.0030664250007248484
   LOF : fini
cancerS  : fini
donneurs  : départ
   IForest : départ
AUC : 0.7805672089358378 RAPPEL : 0.7719422500681014 TEMPS : 2.8159428920007485
   IForest : fini
   LOF : départ


### Function Benchmark 2, where we do for each algo all the datasets first

In [10]:
def create_checkpoint_csv2(datasets, auc_csv, recall_csv,time_csv):
    os.makedirs("./checkpoints", exist_ok=True) # Create the repertories 
    dataset_names = [b.name for b in datasets]
    cols = ["algorithm"] + dataset_names

    # write empty CSVs with that header (mode="w" to overwrite)
    pd.DataFrame(columns=cols).to_csv(   auc_csv,    mode="w", index=False)
    pd.DataFrame(columns=cols).to_csv(   recall_csv, mode="w", index=False)
    pd.DataFrame(columns=cols).to_csv(   time_csv,   mode="w", index=False)

In [11]:
def benchmark_algo_dataset(datasets, algorithms, auc_csv="./checkpoints/auc_checkpoint.csv", recall_csv="./checkpoints/recall_checkpoint.csv",
              time_csv="./checkpoints/time_checkpoint.csv"):
    """
    Run a benchmark of multiple algorithms on multiple datasets.
    
     datasets : list of Bunch
            Each Bunch must have attributes:
              - name : dataset name
              - data : feature data
              - target : ground truth labels
        algorithms : list of tuples
            Each tuple is (algo_name, runner_function, params_dict):
              - algo_name : descriptive name of the algorithm
              - runner_function : function(bunch, params) - > dict
              - params_dict: parameters to pass to the runner


    Returns
    -------
    df_auc : pandas.DataFrame
        AUC scores for each (dataset × algorithm).
    df_recall : pandas.DataFrame
        Recall scores for each (dataset × algorithm).
    df_time : pandas.DataFrame
        Execution time (seconds) for each (dataset × algorithm).
    """

    create_checkpoint_csv2(datasets, auc_csv, recall_csv, time_csv)
    auc_list, recall_list, time_list = [], [], []
    
    for algo_name, algo_runner, params in algorithms:
        print(algo_name, " : départ")
        auc_info, recall_info, time_info    = {"algorithm": algo_name}, {"algorithm": algo_name},{"algorithm": algo_name}        
        
        for bunch in datasets:
        
            print("  ", bunch.name, ": départ")
            res = algo_runner(bunch, params) 
            
            auc_info[bunch.name]    = res["auc"]
            recall_info[bunch.name] = res["recall"]
            time_info[bunch.name]   = res["time"]

            print("  ", "AUC :", res['auc'], "RAPPEL :", res['recall'],"TEMPS :", res['time'])
            print("  ", bunch.name, ": fini")
            
        auc_list.append(auc_info)
        recall_list.append(recall_info)
        time_list.append(time_info)
        print(algo_name, " : fini")

        append_dict_to_csv(auc_info,auc_csv) 
        append_dict_to_csv(recall_info,recall_csv)
        append_dict_to_csv(time_info,time_csv)
        
    df_auc    = pd.DataFrame(auc_list)
    df_recall = pd.DataFrame(recall_list)
    df_time   = pd.DataFrame(time_list)
    
    return df_auc, df_recall, df_time

In [12]:
df1, df2, df3 =benchmark_algo_dataset(datasets, [algorithms[0]])

IForest  : départ
   speech : départ
   AUC : 0.46924590163934415 RAPPEL : 0.0 TEMPS : 0.12134188200070639
   speech : fini
   cancerS : départ
   AUC : 0.977310924369748 RAPPEL : 0.8 TEMPS : 0.08650581199981389
   cancerS : fini
   donneurs : départ
   AUC : 0.7769625451673147 RAPPEL : 0.8348951239444293 TEMPS : 2.9444997900027374
   donneurs : fini
   satellite : départ
   AUC : 0.9478872305140963 RAPPEL : 0.8266666666666667 TEMPS : 0.13317388399809715
   satellite : fini
   fraude : départ
   AUC : 0.9495011258696376 RAPPEL : 0.8313008130081301 TEMPS : 1.2327636670015636
   fraude : fini
   http : départ
   AUC : 0.9717553867904851 RAPPEL : 0.9429657794676806 TEMPS : 1.9389586059987778
   http : fini
   shuttle : départ
   AUC : 0.997285962890537 RAPPEL : 0.9817767653758542 TEMPS : 0.2845184979996702
   shuttle : fini
IForest  : fini


### Rank directly in latex 

In [39]:
def add_hlines(latex):
    return latex.replace("\\\\", "\\\\ \\hline")
        
def pandas_to_latex(df_auc, df_recall, df_time):
    latex_auc = add_hlines(df_auc.to_latex(index=False,float_format="%.2f"))
    latex_recall = add_hlines(df_recall.to_latex(index=False,float_format="%.2f"))
    latex_time = add_hlines(df_time.to_latex(index=False,float_format="%.2f"))

    return latex_auc, latex_recall, latex_time

def pandas_to_latex_with_ranks(df):
    print(df)
    # colonne seulement avec des nombres pour la comparaison 
    num_cols = df.select_dtypes(include="number").columns
    print(num_cols)
    # Comparaison 
    ranks = df[num_cols].rank(axis=1, method='dense', ascending=False).astype(int)
    print(ranks)

    # 
    vals = df[num_cols].round(2)
    print(vals)



In [40]:
pandas_to_latex_with_ranks(df1)

  algorithm    speech   cancerS  donneurs  satellite    fraude      http  \
0   IForest  0.469246  0.977311  0.776963   0.947887  0.949501  0.971755   

    shuttle  
0  0.997286  
Index(['speech', 'cancerS', 'donneurs', 'satellite', 'fraude', 'http',
       'shuttle'],
      dtype='object')
   speech  cancerS  donneurs  satellite  fraude  http  shuttle
0       7        2         6          5       4     3        1
   speech  cancerS  donneurs  satellite  fraude  http  shuttle
0    0.47     0.98      0.78       0.95    0.95  0.97      1.0
